#### Chatbot

LLM powered chatbot

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f85b1deccd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f85b1decbe0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, I am Rakshitha, How can I search for AI jobs in singapore?")])

AIMessage(content='Hello Rakshitha, nice to meet you. If you\'re looking for AI jobs in Singapore, here are some steps and resources to help you get started:\n\n1. **Job Boards**:\n\t* LinkedIn: Use the "Jobs" section and filter by location (Singapore), job title (AI-related), and industry (Tech, Research, etc.).\n\t* Indeed: Search for AI jobs in Singapore using keywords like "AI", "Machine Learning", "Deep Learning", etc.\n\t* Glassdoor: Similar to Indeed, use keywords and filter by location.\n2. **Company Websites**:\n\t* Research and visit the careers pages of top tech companies in Singapore, such as:\n\t\t+ Google\n\t\t+ Amazon\n\t\t+ Microsoft\n\t\t+ NVIDIA\n\t\t+ AI-specific startups like Sentieo, DataRobot, and more.\n3. **Industry-Specific Job Boards**:\n\t* AngelList: A platform for startups, including those in AI and Tech.\n\t* We Work Remotely: Although not exclusively for Singapore, you can find remote AI jobs that may be based in Singapore.\n4. **Professional Networks**:\

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, I am Rakshitha, How can I search for AI jobs in singapore?"),
    AIMessage(content="Hello Rakshitha, nice to meet you. If you\'re looking for AI jobs in Singapore, here are some steps and resources to help you get started:\n\n1. **Job Boards**:\n\t* LinkedIn: Use the Jobs section and filter by location (Singapore), job title (AI-related), and industry (Tech, Research, etc.).\n\t* Indeed: Search for AI jobs in Singapore using keywords like AI, Machine Learning, Deep Learning, etc.\n\t"),
    HumanMessage(content="What is my name and What am I currently doing?")
])

AIMessage(content="You introduced yourself as Rakshitha, and we were discussing searching for AI jobs in Singapore. As for what you're currently doing, I don't have any information about your current activities or profession. I'm a conversational AI, and our conversation just started, so I don't have any prior knowledge about you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 170, 'total_tokens': 236, 'completion_time': 0.089953295, 'completion_tokens_details': None, 'prompt_time': 0.010561342, 'prompt_tokens_details': None, 'queue_time': 0.048652898, 'total_time': 0.100514637}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bde67-5122-79d0-a738-9ae7ecbecd99-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 170, 'output_tokens': 66, 'total_tokens': 236})

#### Message History

It keeps track of inputs and outputs of the model, and store them in database.

In [8]:
## setup chain which wraps the model and add in the message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory ## To chain

store = {}

## Function to create session with session ID -> To retrieve chat history for a specific session ID
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

## To interace with LLM based on chat history
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config={"configurable": {"session_id": "chat1"}}

In [11]:
response = with_message_history.invoke([
    HumanMessage(content="Hi, My name is Rakshitha, Currently I am seraching for AI jobs in singapore")],
    config=config
    )

response.content

'Hello Rakshitha, nice to meet you. I\'d be happy to help you with your job search in Singapore.\n\nTo get started, can you please tell me a bit more about your background and experience? This will help me provide more tailored advice and job suggestions.\n\nSome questions to consider:\n\n1. What is your current level of experience in AI or a related field (e.g. machine learning, data science, computer science)?\n2. What specific areas of AI are you interested in (e.g. natural language processing, computer vision, reinforcement learning)?\n3. Do you have any relevant certifications or education (e.g. Bachelor\'s, Master\'s, Ph.D.) in AI or a related field?\n4. What type of job are you looking for (e.g. research, development, engineering, product management)?\n5. Are you open to working for a startup, mid-sized company, or large corporation?\n\nAlso, I can suggest some popular job boards and resources for AI jobs in Singapore:\n\n1. **LinkedIn**: Use specific keywords like "AI Engineer"

In [12]:
response = with_message_history.invoke([
    HumanMessage(content="what is my name?")],
    config=config
    )

response.content

'Your name is Rakshitha. You mentioned it at the beginning of our conversation.'

In [13]:
## Change the config

config={'configurable':{'session_id': "chat2"}}

response = with_message_history.invoke([
    HumanMessage(content="what is my name?")],
    config=config
    )

response.content

"I don't have any information about your name. I'm a text-based AI model, and our conversation just started. If you'd like to share your name with me, I'd be happy to chat with you."

#### Prompt templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a helpful AI assistant. Please try to provide the best answers for all the questions asked."),
        MessagesPlaceholder(variable_name="messages") ## Whatever messages we are giving from human side, it should hold a key as messages
    ]
)

chain = prompt_template|model

In [18]:
chain.invoke({"messages": [HumanMessage(content="Hello, I am a classical dancer")]})

AIMessage(content='How lovely. Classical dance is a beautiful and expressive art form that requires great technique, discipline, and passion. Which type of classical dance do you specialize in - Bharatanatyam, Odissi, Kathak, Kuchipudi, or one of the Western classical dance forms like Ballet or Modern?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 62, 'total_tokens': 125, 'completion_time': 0.137872459, 'completion_tokens_details': None, 'prompt_time': 0.018698063, 'prompt_tokens_details': None, 'queue_time': 0.059848137, 'total_time': 0.156570522}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bde82-4f51-7510-834e-e14e9d195667-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 62, 'output_tokens': 63, 'total_tokens': 125})

In [20]:
## With chat message history
with_chat_history = RunnableWithMessageHistory(chain, get_session_history)

config = {'configurable': {'session_id': "Chat3"}}
response = with_chat_history.invoke(
    [HumanMessage(content="Hello, I am a classical dancer")],
    config=config
)

response.content

"That's wonderful. Classical dance is a beautiful and expressive art form that requires great technique, discipline, and passion. It's a great way to tell stories, convey emotions, and connect with audiences.\n\nAs a classical dancer, you must have spent countless hours perfecting your craft, practicing techniques, and mastering various styles of dance. Which specific style of classical dance do you specialize in, such as ballet, Bharatanatyam, Kathak, or Odissi?\n\nWhat inspired you to take up classical dance, and what keeps you motivated to continue pursuing this art form?"

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a helpful AI assistant. Please try to provide the best answers for all the questions asked in {language}."),
        MessagesPlaceholder(variable_name="messages") ## Whatever messages we are giving from human side, it should hold a key as messages
    ]
)

chain = prompt_template|model

In [22]:
response = chain.invoke({"messages": [HumanMessage(content="Hello, I am a classical dancer")], "language": "Kannada"})
response.content

'ನಮಸ್ಕಾರ! ನೀವು ಕಲಾವಿದರಾಗಿದ್ದೀರಿ. ಸಂಗೀತ ಮತ್ತು ನೃತ್ಯದಲ್ಲಿ ನೀವು ಕೆಲಸ ಮಾಡುತ್ತೀರಾ? ನೀವು ಯಾವ ಪಾದ್ದತಿಯ ನೃತ್ಯದಲ್ಲಿ ತರಬೇತಿ ಪಡೆದಿದ್ದೀರಿ? ಕರ್ನಾಟಕ ಸಂಗೀತದಲ್ಲಿ ನೀವು ಕೆಲಸ ಮಾಡುತ್ತೀರಾ?'

In [24]:
with_chat_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

config = {'configurable':{"session_id": "chat4"}}
response = with_chat_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Rakshitha and I am a classical dancer and singer")], "language": "German"},
    config=config
)

response.content

'Hallo Rakshitha, schön Sie kennenzulernen! Als klassische Tänzerin und Sängerin sind Sie sicherlich sehr begabt und haben eine großartige Leidenschaft für die Künste. Was für eine Art von klassischer Musik und Tanz bevorzugen Sie am meisten?'

#### Manage Conversation History

'trim_messages': It is a helper to reduce how many messages we're sending to the model. It also allows us to specify how many tokens to keep along with other parameters.

In [25]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    allow_partial=False,
    include_system=True,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/opt/conda/envs/myenv/lib/python3.10/site-packages/langchain_core/language_models/base.py:328: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt_template
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "Kannada"
    }
)

response.content

'ನಿಮ್ಮ ಹೆಸರು ಬಾಬ್ ಎಂದಿದೆ. (Your name is Bob.)'

In [27]:
## Wrap with message history

with_chat_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {'configurable': {"session_id": "chat5"}}

response = with_chat_history.invoke(
    {
        "messages": messages + [HumanMessage(content="Whats my name?")],
        "language": "German"
    },
    config=config
)

response.content

'Dein Name ist Bob, oder?'

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f85b1deccd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f85b1decbe0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, I am Rakshitha, How can I search for AI jobs in singapore?")])

AIMessage(content='Hello Rakshitha, nice to meet you. If you\'re looking for AI jobs in Singapore, here are some steps and resources to help you get started:\n\n1. **Job Boards**:\n\t* LinkedIn: Use the "Jobs" section and filter by location (Singapore), job title (AI-related), and industry (Tech, Research, etc.).\n\t* Indeed: Search for AI jobs in Singapore using keywords like "AI", "Machine Learning", "Deep Learning", etc.\n\t* Glassdoor: Similar to Indeed, use keywords and filter by location.\n2. **Company Websites**:\n\t* Research and visit the careers pages of top tech companies in Singapore, such as:\n\t\t+ Google\n\t\t+ Amazon\n\t\t+ Microsoft\n\t\t+ NVIDIA\n\t\t+ AI-specific startups like Sentieo, DataRobot, and more.\n3. **Industry-Specific Job Boards**:\n\t* AngelList: A platform for startups, including those in AI and Tech.\n\t* We Work Remotely: Although not exclusively for Singapore, you can find remote AI jobs that may be based in Singapore.\n4. **Professional Networks**:\

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, I am Rakshitha, How can I search for AI jobs in singapore?"),
    AIMessage(content="Hello Rakshitha, nice to meet you. If you\'re looking for AI jobs in Singapore, here are some steps and resources to help you get started:\n\n1. **Job Boards**:\n\t* LinkedIn: Use the Jobs section and filter by location (Singapore), job title (AI-related), and industry (Tech, Research, etc.).\n\t* Indeed: Search for AI jobs in Singapore using keywords like AI, Machine Learning, Deep Learning, etc.\n\t"),
    HumanMessage(content="What is my name and What am I currently doing?")
])

AIMessage(content="You introduced yourself as Rakshitha, and we were discussing searching for AI jobs in Singapore. As for what you're currently doing, I don't have any information about your current activities or profession. I'm a conversational AI, and our conversation just started, so I don't have any prior knowledge about you.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 170, 'total_tokens': 236, 'completion_time': 0.089953295, 'completion_tokens_details': None, 'prompt_time': 0.010561342, 'prompt_tokens_details': None, 'queue_time': 0.048652898, 'total_time': 0.100514637}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bde67-5122-79d0-a738-9ae7ecbecd99-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 170, 'output_tokens': 66, 'total_tokens': 236})

#### Message History

It keeps track of inputs and outputs of the model, and store them in database.

In [8]:
## setup chain which wraps the model and add in the message history
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory ## To chain

store = {}

## Function to create session with session ID -> To retrieve chat history for a specific session ID
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

## To interace with LLM based on chat history
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config={"configurable": {"session_id": "chat1"}}

In [11]:
response = with_message_history.invoke([
    HumanMessage(content="Hi, My name is Rakshitha, Currently I am seraching for AI jobs in singapore")],
    config=config
    )

response.content

'Hello Rakshitha, nice to meet you. I\'d be happy to help you with your job search in Singapore.\n\nTo get started, can you please tell me a bit more about your background and experience? This will help me provide more tailored advice and job suggestions.\n\nSome questions to consider:\n\n1. What is your current level of experience in AI or a related field (e.g. machine learning, data science, computer science)?\n2. What specific areas of AI are you interested in (e.g. natural language processing, computer vision, reinforcement learning)?\n3. Do you have any relevant certifications or education (e.g. Bachelor\'s, Master\'s, Ph.D.) in AI or a related field?\n4. What type of job are you looking for (e.g. research, development, engineering, product management)?\n5. Are you open to working for a startup, mid-sized company, or large corporation?\n\nAlso, I can suggest some popular job boards and resources for AI jobs in Singapore:\n\n1. **LinkedIn**: Use specific keywords like "AI Engineer"

In [12]:
response = with_message_history.invoke([
    HumanMessage(content="what is my name?")],
    config=config
    )

response.content

'Your name is Rakshitha. You mentioned it at the beginning of our conversation.'

In [13]:
## Change the config

config={'configurable':{'session_id': "chat2"}}

response = with_message_history.invoke([
    HumanMessage(content="what is my name?")],
    config=config
    )

response.content

"I don't have any information about your name. I'm a text-based AI model, and our conversation just started. If you'd like to share your name with me, I'd be happy to chat with you."

#### Prompt templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a helpful AI assistant. Please try to provide the best answers for all the questions asked."),
        MessagesPlaceholder(variable_name="messages") ## Whatever messages we are giving from human side, it should hold a key as messages
    ]
)

chain = prompt_template|model

In [18]:
chain.invoke({"messages": [HumanMessage(content="Hello, I am a classical dancer")]})

AIMessage(content='How lovely. Classical dance is a beautiful and expressive art form that requires great technique, discipline, and passion. Which type of classical dance do you specialize in - Bharatanatyam, Odissi, Kathak, Kuchipudi, or one of the Western classical dance forms like Ballet or Modern?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 62, 'total_tokens': 125, 'completion_time': 0.137872459, 'completion_tokens_details': None, 'prompt_time': 0.018698063, 'prompt_tokens_details': None, 'queue_time': 0.059848137, 'total_time': 0.156570522}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bde82-4f51-7510-834e-e14e9d195667-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 62, 'output_tokens': 63, 'total_tokens': 125})

In [20]:
## With chat message history
with_chat_history = RunnableWithMessageHistory(chain, get_session_history)

config = {'configurable': {'session_id': "Chat3"}}
response = with_chat_history.invoke(
    [HumanMessage(content="Hello, I am a classical dancer")],
    config=config
)

response.content

"That's wonderful. Classical dance is a beautiful and expressive art form that requires great technique, discipline, and passion. It's a great way to tell stories, convey emotions, and connect with audiences.\n\nAs a classical dancer, you must have spent countless hours perfecting your craft, practicing techniques, and mastering various styles of dance. Which specific style of classical dance do you specialize in, such as ballet, Bharatanatyam, Kathak, or Odissi?\n\nWhat inspired you to take up classical dance, and what keeps you motivated to continue pursuing this art form?"

In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a helpful AI assistant. Please try to provide the best answers for all the questions asked in {language}."),
        MessagesPlaceholder(variable_name="messages") ## Whatever messages we are giving from human side, it should hold a key as messages
    ]
)

chain = prompt_template|model

In [22]:
response = chain.invoke({"messages": [HumanMessage(content="Hello, I am a classical dancer")], "language": "Kannada"})
response.content

'ನಮಸ್ಕಾರ! ನೀವು ಕಲಾವಿದರಾಗಿದ್ದೀರಿ. ಸಂಗೀತ ಮತ್ತು ನೃತ್ಯದಲ್ಲಿ ನೀವು ಕೆಲಸ ಮಾಡುತ್ತೀರಾ? ನೀವು ಯಾವ ಪಾದ್ದತಿಯ ನೃತ್ಯದಲ್ಲಿ ತರಬೇತಿ ಪಡೆದಿದ್ದೀರಿ? ಕರ್ನಾಟಕ ಸಂಗೀತದಲ್ಲಿ ನೀವು ಕೆಲಸ ಮಾಡುತ್ತೀರಾ?'

In [24]:
with_chat_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

config = {'configurable':{"session_id": "chat4"}}
response = with_chat_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Rakshitha and I am a classical dancer and singer")], "language": "German"},
    config=config
)

response.content

'Hallo Rakshitha, schön Sie kennenzulernen! Als klassische Tänzerin und Sängerin sind Sie sicherlich sehr begabt und haben eine großartige Leidenschaft für die Künste. Was für eine Art von klassischer Musik und Tanz bevorzugen Sie am meisten?'

#### Manage Conversation History

'trim_messages': It is a helper to reduce how many messages we're sending to the model. It also allows us to specify how many tokens to keep along with other parameters.

In [25]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=100,
    strategy="last",
    token_counter=model,
    allow_partial=False,
    include_system=True,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/opt/conda/envs/myenv/lib/python3.10/site-packages/langchain_core/language_models/base.py:328: UserWarning: Using fallback GPT-2 tokenizer for token counting. Token counts may be inaccurate for non-GPT-2 models. For accurate counts, use a model-specific method if available.
  return len(self.get_token_ids(text))


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs

In [26]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt_template
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "Kannada"
    }
)

response.content

'ನಿಮ್ಮ ಹೆಸರು ಬಾಬ್ ಎಂದಿದೆ. (Your name is Bob.)'

In [27]:
## Wrap with message history

with_chat_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {'configurable': {"session_id": "chat5"}}

response = with_chat_history.invoke(
    {
        "messages": messages + [HumanMessage(content="Whats my name?")],
        "language": "German"
    },
    config=config
)

response.content

'Dein Name ist Bob, oder?'